In [71]:
import json
import hanlp
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import pandas as pd
from itertools import combinations

In [4]:
print(hanlp.pretrained.ALL.keys())

dict_keys(['SIGHAN2005_PKU_CONVSEG', 'SIGHAN2005_MSR_CONVSEG', 'CTB6_CONVSEG', 'PKU_NAME_MERGED_SIX_MONTHS_CONVSEG', 'LARGE_ALBERT_BASE', 'SIGHAN2005_PKU_BERT_BASE_ZH', 'COARSE_ELECTRA_SMALL_ZH', 'FINE_ELECTRA_SMALL_ZH', 'CTB9_TOK_ELECTRA_SMALL', 'CTB9_TOK_ELECTRA_BASE', 'CTB9_TOK_ELECTRA_BASE_CRF', 'MSR_TOK_ELECTRA_BASE_CRF', 'KYOTO_EVAHAN_TOK_LZH', 'UD_TOK_MMINILMV2L6', 'UD_TOK_MMINILMV2L12', 'CTB5_BIAFFINE_DEP_ZH', 'CTB7_BIAFFINE_DEP_ZH', 'CTB9_DEP_ELECTRA_SMALL', 'PMT1_DEP_ELECTRA_SMALL', 'CTB9_UDC_ELECTRA_SMALL', 'PTB_BIAFFINE_DEP_EN', 'SEMEVAL16_NEWS_BIAFFINE_ZH', 'SEMEVAL16_TEXT_BIAFFINE_ZH', 'SEMEVAL16_ALL_ELECTRA_SMALL_ZH', 'SEMEVAL15_PAS_BIAFFINE_EN', 'SEMEVAL15_PSD_BIAFFINE_EN', 'SEMEVAL15_DM_BIAFFINE_EN', 'GLOVE_6B_50D', 'GLOVE_6B_100D', 'GLOVE_6B_200D', 'GLOVE_6B_300D', 'GLOVE_840B_300D', 'CTB5_POS_RNN', 'CTB5_POS_RNN_FASTTEXT_ZH', 'CTB9_POS_ALBERT_BASE', 'CTB9_POS_ELECTRA_SMALL_TF', 'CTB9_POS_ELECTRA_SMALL', 'CTB9_POS_RADICAL_ELECTRA_SMALL', 'C863_POS_ELECTRA_SMALL', 'PKU

In [68]:
segmenter = hanlp.load('CTB9_TOK_ELECTRA_BASE_CRF')

# 중국어 불용어 리스트 (예제, 필요시 확장 가능)
chinese_stopwords = set([
    "的", "了", "在", "是", "和", "与", "也", "有", "对", "就", "以", "将", "要",
    "但", "其", "而", "此", "我们", "他们", "你们", "可以", "但是", "这样", "这个",
    "其中", "其中之一", "包括", "根据", "由于", "通过", "此外", "同时",
    "记者", "新闻", "媒体", "采访", "发表", "宣布", "透露", "介绍", "报道称",
    "指出", "强调", "证实", "承认", "认为", "评价", "评论", "提到", "解释", "分析",
    "总结", "预测", "预计", "关注", "反映", "说明", "进一步",
    "今天", "昨天", "明天", "日前", "近日", "本周", "上周", "下周", "目前",
    "现在", "过去", "未来", "今年", "去年", "明年", "此前", "随后", "当地",
    "政府", "机构", "相关", "部门", "官员", "领导人", "代表", "发言人", "部长", "事务", "委员会",
    "一", "二", "三", "这", "不", "可能", "都", "才", "可", "一直", "到", "如果", "将",
    "带来", "十分", "称", "个", "月", "次", "因", "因此", "认为", "如果", "最", "即",
    "当天", "因为", "曾", "号", "第", "等", "相当", "个", "两", "很", "所以", "各种",
    "从而", "仍", "以", "为了", "以及", "据", "并", "过", "几", "号", "立即", "相关",
    "等", "着", "于", "为", "说", "却", "使", "国", "还", "带来", "来说", "而", "为",
    "至", "可以", "很", "会", "除了", "被", "外", "也", "若", "更", "已经", "已", "称",
    "大大", "不仅", "能够", "近日", "并", "再", "一些", "明确", "作为", "向", "于",
    "表示", "以及"
])

In [69]:
# JSON 파일 로드
file_path = "../data/final_0307.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 결과 저장 리스트
processed_data = []
tokenized_texts = []  # TF 및 TF-IDF 분석용

# 연도 및 분기별 데이터 저장
yearly_data = {}
quarterly_data = {}

# 데이터 처리
for item in data:
    content = item.get("cleaned_content", "")
    date_str = item.get("date")  # 날짜 정보
    quarter = item.get("quarter")  # 분기 정보

    if not date_str or not quarter:
        continue  # 날짜 또는 분기 정보 없는 데이터 제외

    # 날짜에서 연도 추출
    try:
        year = pd.to_datetime(date_str).year
    except:
        continue  # 유효하지 않은 날짜 제외

    # 한자 및 필요 문자만 유지
    clean_text = re.sub(r"[^\u4e00-\u9fff0-9]", "", content)

    # HanLP 토큰화
    tokens = segmenter(clean_text)

    # 불용어 제거
    filtered_tokens = [word for word in tokens if word not in chinese_stopwords]

    # 연도별 데이터 저장
    if year not in yearly_data:
        yearly_data[year] = []
    yearly_data[year].append(filtered_tokens)

    # 분기별 데이터 저장
    if quarter not in quarterly_data:
        quarterly_data[quarter] = []
    quarterly_data[quarter].append(filtered_tokens)

In [60]:
print("\n📌 분기별 중요 키워드 분석 (TF-IDF)")

for quarter, token_lists in quarterly_data.items():
    tokenized_texts = [" ".join(tokens) for tokens in token_lists]

    # TF-IDF 벡터라이저 적용
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(tokenized_texts)

    # 단어 리스트
    feature_names = vectorizer.get_feature_names_out()

    print(f"\n📅 {quarter}분기 중요 키워드 (TF-IDF 상위 10개):")

    # 모든 문서의 평균 TF-IDF 점수 계산
    mean_tfidf_scores = tfidf_matrix.mean(axis=0).A1
    sorted_indices = mean_tfidf_scores.argsort()[::-1]  # 중요도 높은 순으로 정렬
    top_keywords = [feature_names[idx] for idx in sorted_indices[:10]]

    print(top_keywords)


📌 분기별 중요 키워드 분석 (TF-IDF)

📅 2006Q3분기 중요 키워드 (TF-IDF 상위 10개):
['朝鲜', '导弹', '日本', '发射', '制裁', '美国', '问题', '议案', '防御', '系统']

📅 2007Q1분기 중요 키워드 (TF-IDF 상위 10개):
['会谈', '朝鲜', '美国', '关系', '问题', '正常化', '金桂冠', '希尔', '谈判', '汇业']

📅 2008Q3분기 중요 키워드 (TF-IDF 상위 10개):
['朝鲜', '情报', '韩国', '美国', '可能', '国家', '侦察机', '侦察', '日本', '以及']

📅 2009Q1분기 중요 키워드 (TF-IDF 상위 10개):
['朝鲜', '导弹', '美国', '7日', '韩国', '愿意', '博斯沃思', '对话', '可能', '试射']

📅 2009Q2분기 중요 키워드 (TF-IDF 상위 10개):
['朝鲜', '发射', '声明', '安理会', '中国', '卫星', '美国', '日本', '表示', '葛瑞格森']

📅 2009Q3분기 중요 키워드 (TF-IDF 상위 10개):
['日本', '导弹', '朝鲜', '发射', '行为', '报道', '飞行', '挑衅', '严重', '弹道']

📅 2010Q1분기 중요 키워드 (TF-IDF 상위 10개):
['演习', '联合', '韩国', '参加', '军事', '军演', '美军', '金色', '报道', '眼镜蛇']

📅 2010Q3분기 중요 키워드 (TF-IDF 상위 10개):
['军演', '中国', '美国', '联合', '演习', '日本', '韩国', '朝鲜', '进行', '军事']

📅 2010Q4분기 중요 키워드 (TF-IDF 상위 10개):
['朝鲜', '日本', '美国', '会谈', '中国', '报道', '6日', '韩国', '外长', '半岛']

📅 2011Q1분기 중요 키워드 (TF-IDF 상위 10개):
['朝鲜', '军演', '日本', '韩国', '美国', '军事', '协定', '合作', '战略', '

In [77]:
print("\n📌 연도별 중요 키워드 분석 (TF-IDF)")

# 연도별 키워드 및 N-그램 저장 딕셔너리
yearly_bigrams = {}
yearly_trigrams = {}

for year, token_lists in yearly_data.items():
    tokenized_texts = [" ".join(tokens) for tokens in token_lists]

    # TF-IDF 벡터라이저 적용
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(tokenized_texts)

    # 단어 리스트
    feature_names = vectorizer.get_feature_names_out()

    print(f"\n📅 {year}년 중요 키워드 (TF-IDF 상위 10개):")

    # 모든 문서의 평균 TF-IDF 점수 계산
    mean_tfidf_scores = tfidf_matrix.mean(axis=0).A1
    sorted_indices = mean_tfidf_scores.argsort()[::-1]  # 중요도 높은 순으로 정렬
    top_keywords = [feature_names[idx] for idx in sorted_indices[:10]]

    print(top_keywords)

    bigram_counter = Counter()
    trigram_counter = Counter()

    # 연도별 전체 기사에서 키워드 추출
    for tokens in token_lists:
        # 2-그램, 3-그램 조합 생성
        if len(tokens) >= 2:
            bigram_counter.update(combinations(tokens, 2))
        if len(tokens) >= 3:
            trigram_counter.update(combinations(tokens, 3))

    # 연도별 N-그램 저장
    yearly_bigrams[year] = bigram_counter.most_common(10)  # 상위 10개 2-그램
    yearly_trigrams[year] = trigram_counter.most_common(10)  # 상위 10개 3-그램


📌 연도별 중요 키워드 분석 (TF-IDF)

📅 2006년 중요 키워드 (TF-IDF 상위 10개):
['朝鲜', '导弹', '日本', '发射', '制裁', '计划', '防御', '问题', '推动', '议案']

📅 2007년 중요 키워드 (TF-IDF 상위 10개):
['会谈', '朝鲜', '美国', '关系', '问题', '正常化', '金桂冠', '希尔', '纽约', '银行']

📅 2008년 중요 키워드 (TF-IDF 상위 10개):
['朝鲜', '情报', '韩国', '美国', '侦察机', '日本', '国家', '侦察', '金正日', '这些']

📅 2009년 중요 키워드 (TF-IDF 상위 10개):
['朝鲜', '导弹', '发射', '美国', '声明', '日本', '中国', '韩国', '安理会', '卫星']

📅 2010년 중요 키워드 (TF-IDF 상위 10개):
['朝鲜', '日本', '中国', '美国', '军演', '韩国', '会谈', '演习', '联合', '报道']

📅 2011년 중요 키워드 (TF-IDF 상위 10개):
['朝鲜', '日本', '军演', '军事', '韩国', '美国', '签署', '问题', '协定', '合作']


MemoryError: 

In [73]:
# 📌 3. 연도별 N-그램 결과 출력
print("\n✅ 연도별 상위 10개 2-그램:")
for year, bigrams in yearly_bigrams.items():
    print(f"\n📅 {year}년:")
    print(bigrams)

print("\n✅ 연도별 상위 10개 3-그램:")
for year, trigrams in yearly_trigrams.items():
    print(f"\n📅 {year}년:")
    print(trigrams)


✅ 연도별 상위 10개 2-그램:

📅 2006년:
[(('朝鲜', '导弹'), 1), (('朝鲜', '日本'), 1), (('朝鲜', '发射'), 1), (('朝鲜', '制裁'), 1), (('朝鲜', '计划'), 1), (('朝鲜', '防御'), 1), (('朝鲜', '问题'), 1), (('朝鲜', '推动'), 1), (('朝鲜', '议案'), 1), (('导弹', '日本'), 1)]

📅 2007년:
[(('会谈', '朝鲜'), 1), (('会谈', '美国'), 1), (('会谈', '关系'), 1), (('会谈', '问题'), 1), (('会谈', '正常化'), 1), (('会谈', '金桂冠'), 1), (('会谈', '希尔'), 1), (('会谈', '纽约'), 1), (('会谈', '银行'), 1), (('朝鲜', '美国'), 1)]

📅 2008년:
[(('朝鲜', '情报'), 1), (('朝鲜', '韩国'), 1), (('朝鲜', '美国'), 1), (('朝鲜', '侦察机'), 1), (('朝鲜', '日本'), 1), (('朝鲜', '国家'), 1), (('朝鲜', '侦察'), 1), (('朝鲜', '金正日'), 1), (('朝鲜', '这些'), 1), (('情报', '韩国'), 1)]

📅 2009년:
[(('朝鲜', '导弹'), 1), (('朝鲜', '发射'), 1), (('朝鲜', '美国'), 1), (('朝鲜', '声明'), 1), (('朝鲜', '日本'), 1), (('朝鲜', '中国'), 1), (('朝鲜', '韩国'), 1), (('朝鲜', '安理会'), 1), (('朝鲜', '卫星'), 1), (('导弹', '发射'), 1)]

📅 2010년:
[(('朝鲜', '日本'), 1), (('朝鲜', '中国'), 1), (('朝鲜', '美国'), 1), (('朝鲜', '军演'), 1), (('朝鲜', '韩国'), 1), (('朝鲜', '会谈'), 1), (('朝鲜', '演习'), 1), (('朝鲜', '联合'), 1), (('朝鲜', 

In [74]:
# 📌 4. 데이터프레임 변환 (시각화 및 저장을 위해)
bigram_df = pd.DataFrame(
    [(year, " → ".join(pair), count) for year, bigrams in yearly_bigrams.items() for pair, count in bigrams],
    columns=["year", "bigram", "count"]
)

trigram_df = pd.DataFrame(
    [(year, " → ".join(pair), count) for year, trigrams in yearly_trigrams.items() for pair, count in trigrams],
    columns=["year", "trigram", "count"]
)

In [76]:
# 📌 5. CSV 파일로 저장
bigram_csv_path = "../data/yearly_bigram.csv"
trigram_csv_path = "../data/yearly_trigram.csv"

bigram_df.to_csv(bigram_csv_path, index=False, encoding="utf-8-sig")
trigram_df.to_csv(trigram_csv_path, index=False, encoding="utf-8-sig")

print(f"\n✅ 연도별 2-그램 CSV 저장 완료: {bigram_csv_path}")
print(f"✅ 연도별 3-그램 CSV 저장 완료: {trigram_csv_path}")

# 📌 6. JSON 파일로 저장
output_json_path = "../data/yearly_keyword_ngrams.json"
ngrams_data = {
    "bigrams": yearly_bigrams,
    "trigrams": yearly_trigrams,
}
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(ngrams_data, f, indent=4, ensure_ascii=False)

print(f"\n✅ 연도별 키워드 기반 N-그램 분석 결과 저장 완료: {output_json_path}")


✅ 연도별 2-그램 CSV 저장 완료: ../data/yearly_bigram.csv
✅ 연도별 3-그램 CSV 저장 완료: ../data/yearly_trigram.csv

✅ 연도별 키워드 기반 N-그램 분석 결과 저장 완료: ../data/yearly_keyword_ngrams.json


In [59]:

print("\n📌 분기별 단어 빈도 분석 (TF)")

for quarter, token_lists in quarterly_data.items():
    all_words = [word for tokens in token_lists for word in tokens]
    word_freq = Counter(all_words)

    print(f"\n📅 {quarter}분기 가장 많이 등장한 키워드 (TF 상위 10개):")
    for word, freq in word_freq.most_common(10):
        print(f"{word}: {freq}회")


📌 분기별 단어 빈도 분석 (TF)

📅 2006Q3분기 가장 많이 등장한 키워드 (TF 상위 10개):
朝鲜: 10회
导弹: 7회
日本: 6회
日: 5회
发射: 4회
美: 4회
制裁: 3회
次: 3회
国际: 2회
安理会: 2회

📅 2007Q1분기 가장 많이 등장한 키워드 (TF 상위 10개):
会谈: 16회
朝鲜: 13회
朝: 13회
美国: 12회
美: 12회
关系: 10회
问题: 7회
两: 6회
国: 6회
正常化: 6회

📅 2008Q3분기 가장 많이 등장한 키워드 (TF 상위 10개):
朝鲜: 53회
情报: 33회
三: 17회
国: 17회
韩国: 14회
核: 13회
美国: 11회
美: 11회
国家: 8회
侦察机: 8회

📅 2009Q1분기 가장 많이 등장한 키워드 (TF 상위 10개):
朝鲜: 61회
导弹: 61회
韩国: 17회
可能: 16회
美: 15회
试射: 15회
公里: 13회
美国: 12회
韩: 11회
日: 11회

📅 2009Q2분기 가장 많이 등장한 키워드 (TF 상위 10개):
朝鲜: 66회
发射: 28회
说: 24회
美国: 21회
声明: 19회
日: 18회
韩国: 17회
日本: 17회
安理会: 16회
并: 15회

📅 2009Q3분기 가장 많이 등장한 키워드 (TF 상위 10개):
导弹: 6회
日本: 6회
朝鲜: 5회
称: 4회
发射: 4회
报道: 3회
据: 3회
行为: 3회
枚: 2회
安全: 2회

📅 2010Q1분기 가장 많이 등장한 키워드 (TF 상위 10개):
演习: 7회
联合: 6회
国: 6회
韩国: 5회
参加: 5회
军事: 5회
军演: 4회
报道: 3회
美军: 3회
表示: 3회

📅 2010Q3분기 가장 많이 등장한 키워드 (TF 상위 10개):
美: 45회
韩: 44회
军演: 40회
美国: 28회
联合: 27회
中国: 26회
次: 25회
演习: 23회
一: 22회
日本: 19회

📅 2010Q4분기 가장 많이 등장한 키워드 (TF 상위 10개):
朝鲜: 52회
韩: 49회
美: 47회
日本: 46회
美国: 44회
日: 36회

In [61]:
print("\n📌 연도별 단어 빈도 분석 (TF)")

for year, token_lists in yearly_data.items():
    all_words = [word for tokens in token_lists for word in tokens]
    word_freq = Counter(all_words)

    print(f"\n📅 {year}년 가장 많이 등장한 키워드 (TF 상위 10개):")
    for word, freq in word_freq.most_common(10):
        print(f"{word}: {freq}회")


📌 연도별 단어 빈도 분석 (TF)

📅 2006년 가장 많이 등장한 키워드 (TF 상위 10개):
朝鲜: 10회
导弹: 7회
日本: 6회
日: 5회
发射: 4회
美: 4회
制裁: 3회
次: 3회
国际: 2회
安理会: 2회

📅 2007년 가장 많이 등장한 키워드 (TF 상위 10개):
会谈: 16회
朝鲜: 13회
朝: 13회
美国: 12회
美: 12회
关系: 10회
问题: 7회
两: 6회
国: 6회
正常化: 6회

📅 2008년 가장 많이 등장한 키워드 (TF 상위 10개):
朝鲜: 53회
情报: 33회
三: 17회
国: 17회
韩国: 14회
核: 13회
美国: 11회
美: 11회
国家: 8회
侦察机: 8회

📅 2009년 가장 많이 등장한 키워드 (TF 상위 10개):
朝鲜: 132회
导弹: 68회
发射: 41회
韩国: 35회
美国: 33회
日本: 30회
日: 29회
说: 28회
美: 26회
可能: 22회

📅 2010년 가장 많이 등장한 키워드 (TF 상위 10개):
韩: 94회
美: 93회
美国: 73회
朝鲜: 68회
日本: 66회
军演: 65회
中国: 58회
国: 55회
日: 47회
一: 47회

📅 2011년 가장 많이 등장한 키워드 (TF 상위 10개):
韩: 97회
美国: 89회
日: 86회
朝鲜: 69회
国: 66회
韩国: 56회
美: 54회
日本: 51회
一: 49회
军事: 48회

📅 2012년 가장 많이 등장한 키워드 (TF 상위 10개):
朝鲜: 525회
美国: 486회
日本: 393회
日: 347회
韩国: 346회
韩: 323회
美: 278회
中国: 258회
一: 252회
这: 230회

📅 2013년 가장 많이 등장한 키워드 (TF 상위 10개):
朝鲜: 943회
美国: 435회
日本: 409회
导弹: 387회
韩国: 374회
韩: 358회
日: 332회
一: 302회
不: 270회
美: 268회

📅 2014년 가장 많이 등장한 키워드 (TF 상위 10개):
日本: 556회
韩: 529회
美国: 522회
日: 507회
韩国: 49